In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn
from sklearn import linear_model
# from __future__ import division

from scipy import stats
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm
from statsmodels.stats.multicomp import pairwise_tukeyhsd

%matplotlib inline
seaborn.set()
plt.rcParams['font.sans-serif']=['SimHei']
plt.rcParams['axes.unicode_minus'] = False

G:\Anaconda3\lib\site-packages\sklearn\linear_model\least_angle.py:35: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  eps=np.finfo(np.float).eps,
G:\Anaconda3\lib\site-packages\sklearn\linear_model\least_angle.py:597: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  eps=np.finfo(np.float).eps, copy_X=True, fit_path=True,
G:\Anaconda3\lib\site

In [2]:
# 确诊数据导入和初步处理
f_confirm = open(r'各国R0.csv',encoding = "utf-8")
data_confirm = pd.read_csv(f_confirm)
# 处理时间格式
data_confirm['时间'] = pd.to_datetime(data_confirm['时间'])

In [3]:
# 导入政策详细信息
f_policy = open(r'政策详细信息_new.csv',encoding='utf-8')
data_policy = pd.read_csv(f_policy)
# 处理时间格式
data_policy['Date'] = data_policy['Date'].astype(str)
data_policy['Date'] = pd.to_datetime(data_policy['Date'])

In [4]:
# 建立字典，对应政策代号及其中英文名
dic_policyEng = dict()
dic_policyChi = dict()

# 封闭措施
dic_policyEng['C1'] = 'School closing'
dic_policyChi['C1'] = '关闭学校'
dic_policyEng['C2'] = 'Workplace closing'
dic_policyChi['C2'] = '关闭工作场所'
dic_policyEng['C3'] = 'Cancel public events'
dic_policyChi['C3'] = '取消公共活动'
dic_policyEng['C4'] = 'Restrictions on gatherings'
dic_policyChi['C4'] = '限制集会'
dic_policyEng['C5'] = 'Close public transport'
dic_policyChi['C5'] = '关闭公共交通'
dic_policyEng['C6'] = 'Stay at home requirements'
dic_policyChi['C6'] = '居家要求'
dic_policyEng['C7'] = 'Restrictions on internal movement'
dic_policyChi['C7'] = '国内移动限制'
dic_policyEng['C8'] = 'International travel controls'
dic_policyChi['C8'] = '国际旅行控制'

# 经济政策
dic_policyEng['E1'] = 'Income support'
dic_policyChi['E1'] = '收入支持'
dic_policyEng['E2'] = 'Debt/contract relief'
dic_policyChi['E2'] = '债务/合同放宽'
dic_policyEng['E3'] = 'Fiscal measures'
dic_policyChi['E3'] = '财政措施'
dic_policyEng['E4'] = 'International support'
dic_policyChi['E4'] = '给予国际支持'

# 卫生系统政策
dic_policyEng['H1'] = 'Public information campaigns'
dic_policyChi['H1'] = '公共信息宣传'
dic_policyEng['H2'] = 'Testing policy'
dic_policyChi['H2'] = '检测政策'
dic_policyEng['H3'] = 'Contact tracing'
dic_policyChi['H3'] = '接触者追踪'
dic_policyEng['H4'] = 'Emergency investment in healthcare'
dic_policyChi['H4'] = '医疗保健紧急投资'
dic_policyEng['H5'] = 'Investment in vaccines'
dic_policyChi['H5'] = '疫苗投资'
dic_policyEng['H6'] = 'Facial Coverings'
dic_policyChi['H6'] = '口罩政策'
dic_policyEng['H7'] = 'Vaccination Policy'
dic_policyChi['H7'] = '疫苗接种政策'

ls_code = list(dic_policyEng.keys())
for code in ['E3','E4','H4','H5','H7']:
    ls_code.remove(code)
ls_code

['C1',
 'C2',
 'C3',
 'C4',
 'C5',
 'C6',
 'C7',
 'C8',
 'E1',
 'E2',
 'H1',
 'H2',
 'H3',
 'H6']

In [5]:
# 取政策引入的第一天和政策放宽的第一天
def get1stDate(code,condition):
    # 获取第一天的索引
    timedelta1 = pd.to_timedelta('7D')
    timedelta2 = pd.to_timedelta('27D')
    startDate = pd.to_datetime('2020-01-22')
    endDate = pd.to_datetime('2020-12-31')
    if condition in ['Introduce','Lift']:
        subdata = data_policy[data_policy[code+condition]==1]
    else:
        subdata = data_policy[data_policy[code+'28']==0]
    subdata = subdata[(subdata['Date']- startDate >= timedelta1)&(endDate - subdata['Date'] >= timedelta2)]
    
    subdata = subdata[subdata['R0']!=0]
    index_day1 = subdata.index

    # 构造各政策所需的数据集
    data_code = pd.DataFrame()
    # 国家名和第一天日期
    data_code['CountryName'] = data_policy['CountryName'].loc[index_day1]
    data_code['DateDay1'] = data_policy['Date'].loc[index_day1]
    data_code = data_code.reset_index(drop=True)
    # “第0天”的R0
    ls_day0_R0 = []
    for index in list(index_day1):
        ls_day0_R0.append(data_policy['R0'].loc[index-7:index].mean())
    Series_day0_R0 = pd.Series(ls_day0_R0)
    array_day0_R0 = np.array(Series_day0_R0)
    data_code['day0_R0'] = Series_day0_R0
    # 第1到第28天的R0和R0_Ratio
    for i in range(28):
        index = index_day1 + i
        Series_dayi_R0 = data_policy['R0'].loc[index]
        Series_dayi_R0 = Series_dayi_R0.reset_index(drop=True)
        data_code['day{}_R0'.format(i+1)] = Series_dayi_R0

    # 去除R0值过大的行
    for i in range(29):
        data_code = data_code[(data_code['day{}_R0'.format(i)]>0.85)&(data_code['day{}_R0'.format(i)]<=2)]
    '''data_code = data_code[data_code['day0_R0']>0.85]
    data_code = data_code.reset_index(drop=True)
    subdata1 = data_code.iloc[:,:2]
    subdata2 = data_code.iloc[:,2:]
    # print(((subdata2<=2)&(subdata2>=-2)).any(axis=1))
    subdata2 = subdata2.loc[(abs(subdata2)<=2).any(axis=1),:]
    # subdata2 = subdata2.loc[((subdata2<=2)&(subdata2>=-2)).any(axis=1),:]
    # subdata2 = subdata2.loc[(subdata2>=-2).any(axis=1),:]
    subindex = subdata2.index
    subdata1 = subdata1.loc[subindex]
    data_code = subdata1.join(subdata2)
    data_code = data_code.reset_index(drop=True)'''

    # 累乘R0查看累积效应
    for i in range(28):
        data_code['day{}_R0_Ratio'.format(i+1)] = data_code['day{}_R0'.format(i+1)]/data_code['day0_R0']
        if i == 0:
            data_code['day{}_TimesR0'.format(i+1)] = data_code['day{}_R0_Ratio'.format(i+1)]
        else:
            data_code['day{}_TimesR0'.format(i+1)] = data_code['day{}_R0_Ratio'.format(i+1)] * data_code['day{}_TimesR0'.format(i)]

        '''array_dayi_R0 = np.array(Series_dayi_R0)
        array_dayi_R0Ratio = (array_dayi_R0/(1+array_day0_R0))-0.5
        Series_day_R0Ratio = pd.DataFrame(array_dayi_R0Ratio)
        data_code['day{}_R0Ratio'.format(i+1)] = Series_day_R0Ratio'''

        '''if Series_day_R0Ratio[np.isinf(Series_day_R0Ratio)].dropna(how='any').shape[0] != 0:
            print(code,condition,'day')'''
        #print(code,condition,'day',i+1,Series_day_R0Ratio.max(),Series_day_R0Ratio.min())
    # print(code,' ',condition,data_code.shape[0])
    data_code.to_csv(code+'\\'+code+'_'+condition+'.csv',index=False,sep=',',encoding="utf_8_sig")
    return data_code

In [6]:
for code in ls_code:
    for condition in ['Introduce','Lift','NoSuchPolicy']:
        get1stDate(code,condition)

In [7]:
for i in range(8):
    string1 = 'C{}\\C{}_Introduce.csv'.format(i+1,i+1)
    string2 = 'C{}\\C{}_Lift.csv'.format(i+1,i+1)
    string3 = 'C{}\\C{}_NoSuchPolicy.csv'.format(i+1,i+1)
    f_intro = open(string1,encoding='utf-8')
    data_intro = pd.read_csv(f_intro)
    f_lift = open(string2,encoding='utf-8')
    data_lift = pd.read_csv(f_lift)
    f_nop = open(string3,encoding='utf-8')
    data_nop = pd.read_csv(f_nop)
    print(i+1)
    print(data_intro['day28_R0_Ratio'].mean())
    print(data_nop['day28_R0_Ratio'].mean())
    print(data_lift['day28_R0_Ratio'].mean())

1
0.8795630533433192
1.009274389963771
1.0049703746434073
2
0.8745833997043975
1.007757942081302
1.0018902380237649
3
0.901161379020775
1.0159839565472149
1.0081453798981233
4
0.9197271343518071
0.997858890120767
1.0059801857397843
5
0.8952445562246891
0.9921479275844194
0.9920468560758987
6
0.8963614619719572
1.0009570764172289
0.9880184615730833
7
0.8982875743118162
1.0037996473959963
0.9943474120109663
8
0.9318181583883598
1.005463923021254
1.0045580741332425


In [8]:
data1 = pd.DataFrame([[1]],columns=['A'])
data2 = pd.DataFrame([[2]],columns=['B'])
data3 = data1.join(data2)